# Demand Planning Tool - Production Forecast

This notebook creates a comprehensive demand planning analysis with:
- Historical sales analysis by SKU and channel
- Statistical forecasts through 12/31/2026
- Trend and seasonality analysis
- Product type mix analysis
- Export to Google Sheets

## Setup Instructions
1. Upload your CSV file when prompted
2. Run all cells in order
3. Authenticate with Google when prompted
4. The output will be saved to your Google Drive

In [ ]:
# Install required packages
!pip install gspread oauth2client pandas numpy openpyxl -q

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from google.colab import files
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials

In [ ]:
# Upload your CSV file
print("Please upload your sales data CSV file:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"\nFile '{filename}' uploaded successfully!")

In [ ]:
# Load and explore the data
df = pd.read_csv(filename)

print("Data Overview:")
print(f"Total rows: {len(df):,}")
print(f"Date range: {df['created_date'].min()} to {df['created_date'].max()}")
print(f"\nChannels: {df['orders__source'].unique()}")
print(f"Unique SKUs: {df['products__variants__sku'].nunique()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Prepare data for analysis
df['created_date'] = pd.to_datetime(df['created_date'])
df['year_month'] = df['created_date'].dt.to_period('M')
df['year'] = df['created_date'].dt.year
df['month'] = df['created_date'].dt.month

# Aggregate to monthly level by SKU and channel
monthly_data = df.groupby(['year_month', 'products__variants__sku', 'orders__source'])['quantity'].sum().reset_index()
monthly_data['year_month_str'] = monthly_data['year_month'].astype(str)

# Get SKU details
sku_details = df.groupby('products__variants__sku').agg({
    'products__variants__title': 'first',
    'products__root_product__title': 'first',
    'products__product_type': 'first'
}).reset_index()

print("Data aggregated to monthly level")
print(f"Monthly records: {len(monthly_data):,}")

In [ ]:
# Define forecasting function
def calculate_forecast(historical_values, forecast_periods=11, growth_rate=0.0):
    """
    Calculate forecast using 12-month moving average with seasonal adjustment
    
    Parameters:
    - historical_values: numpy array of historical quantities
    - forecast_periods: number of months to forecast
    - growth_rate: annual growth rate (e.g., 0.10 for 10% growth, -0.05 for 5% decline)
    
    Returns:
    - List of forecasted values
    """
    if len(historical_values) < 12:
        # Not enough history, use simple average
        avg = np.mean(historical_values) if len(historical_values) > 0 else 0
        base_forecast = avg
    else:
        # Calculate 12-month moving average
        base_forecast = np.mean(historical_values[-12:])
    
    # Calculate seasonality indices (last 12 months vs their average)
    if len(historical_values) >= 12:
        recent_12 = historical_values[-12:]
        avg_12 = np.mean(recent_12)
        
        if avg_12 > 0:
            seasonal_indices = recent_12 / avg_12
        else:
            seasonal_indices = np.ones(12)
    else:
        seasonal_indices = np.ones(12)
    
    # Generate forecast with growth applied
    forecasts = []
    monthly_growth = (1 + growth_rate) ** (1/12) - 1  # Convert annual to monthly
    
    for i in range(forecast_periods):
        month_idx = i % 12
        seasonal_factor = seasonal_indices[month_idx]
        
        # Apply growth compounding over time
        growth_factor = (1 + monthly_growth) ** i
        forecast = base_forecast * seasonal_factor * growth_factor
        forecasts.append(max(0, forecast))
    
    return forecasts

print("Forecast function defined")

In [ ]:
# ============================================================================
# TUNE YOUR FORECAST GROWTH RATE HERE
# ============================================================================
# Set the annual growth rate for your forecast
# Examples:
#   0.10  = 10% growth
#   0.05  = 5% growth
#   0.0   = flat (no growth)
#  -0.05  = 5% decline
#  -0.10  = 10% decline

FORECAST_GROWTH_RATE = 0.0  # Change this value to tune growth

print(f"Forecast Growth Rate: {FORECAST_GROWTH_RATE*100:.1f}% annually")
print(f"This equals approximately {((1+FORECAST_GROWTH_RATE)**(1/12)-1)*100:.2f}% per month")

In [ ]:
# Generate forecasts for each channel and SKU
forecast_months = pd.period_range('2026-02', '2026-12', freq='M')
# Only use DTC, Wholesale, and TOTAL (not Kristina Holey)
all_channels = ['Direct-to-Consumer', 'Wholesale', 'TOTAL']

forecast_results = []

for channel in all_channels:
    print(f"Generating forecasts for {channel}...")
    
    # Filter data for this channel
    if channel == 'TOTAL':
        channel_data = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()
    else:
        channel_data = monthly_data[monthly_data['orders__source'] == channel].copy()
    
    # Get unique SKUs
    channel_skus = channel_data['products__variants__sku'].unique()
    
    for sku in channel_skus:
        sku_data = channel_data[channel_data['products__variants__sku'] == sku]
        sku_info = sku_details[sku_details['products__variants__sku'] == sku].iloc[0]
        
        # Get historical data (up to 2026-01)
        historical = sku_data[sku_data['year_month_str'] <= '2026-01'].sort_values('year_month_str')
        hist_values = historical['quantity'].values
        
        # Calculate forecasts
        forecasts = calculate_forecast(hist_values, len(forecast_months), FORECAST_GROWTH_RATE)
        
        # Store results
        for month, forecast_qty in zip(forecast_months, forecasts):
            forecast_results.append({
                'channel': channel,
                'sku': sku,
                'product_name': sku_info['products__variants__title'],
                'product_type': sku_info['products__product_type'],
                'month': str(month),
                'forecast_qty': round(forecast_qty, 1)
            })

forecast_df = pd.DataFrame(forecast_results)
print(f"\nForecasts generated: {len(forecast_df):,} records")

In [ ]:
# Create forecast comparison by aggregation level
print("Creating forecast comparison analysis...\n")

comparison_results = []

for channel in all_channels:
    channel_forecasts = forecast_df[forecast_df['channel'] == channel]
    
    # Level A: Total forecast for channel (sum of all SKU forecasts)
    total_channel_forecast = channel_forecasts['forecast_qty'].sum()
    
    # Level B: Forecast by product type
    product_type_forecasts = channel_forecasts.groupby('product_type')['forecast_qty'].sum().reset_index()
    total_by_product_type = product_type_forecasts['forecast_qty'].sum()
    
    # Level C: Forecast by SKU (already at this level)
    total_by_sku = channel_forecasts.groupby('sku')['forecast_qty'].sum().sum()
    
    comparison_results.append({
        'Channel': channel,
        'A - Total Forecast': round(total_channel_forecast, 0),
        'B - Product Type': round(total_by_product_type, 0),
        'C - Item Level': round(total_by_sku, 0),
        'B vs A Diff': round(total_by_product_type - total_channel_forecast, 2),
        'C vs A Diff': round(total_by_sku - total_channel_forecast, 2)
    })

comparison_df = pd.DataFrame(comparison_results)
print("Forecast Comparison by Aggregation Level:")
print(comparison_df)
print("\nNote: All three levels should show the same total (different aggregation paths)")

In [ ]:
# Create detailed product type forecast breakdown
print("Creating product type forecast breakdown...\n")

product_type_details = []

for channel in all_channels:
    channel_forecasts = forecast_df[forecast_df['channel'] == channel]
    
    # Group by product type
    pt_summary = channel_forecasts.groupby('product_type').agg({
        'forecast_qty': 'sum',
        'sku': 'nunique'
    }).reset_index()
    
    pt_summary.columns = ['Product_Type', 'Total_Forecast_Qty', 'Num_SKUs']
    pt_summary['Channel'] = channel
    pt_summary['Avg_Per_SKU'] = (pt_summary['Total_Forecast_Qty'] / pt_summary['Num_SKUs']).round(1)
    
    # Calculate percentage of channel total
    channel_total = pt_summary['Total_Forecast_Qty'].sum()
    pt_summary['Pct_of_Channel'] = ((pt_summary['Total_Forecast_Qty'] / channel_total) * 100).round(1)
    
    product_type_details.append(pt_summary)

product_type_df = pd.concat(product_type_details, ignore_index=True)
product_type_df = product_type_df[['Channel', 'Product_Type', 'Num_SKUs', 'Total_Forecast_Qty', 'Avg_Per_SKU', 'Pct_of_Channel']]

print("Product Type Forecast Breakdown:")
product_type_df

In [ ]:
# Create summary by channel
summary_by_channel = []

for channel in all_channels:
    # Filter data
    if channel == 'TOTAL':
        channel_monthly = monthly_data.groupby(['year_month_str'])['quantity'].sum().reset_index()
    else:
        channel_monthly = monthly_data[monthly_data['orders__source'] == channel].groupby(['year_month_str'])['quantity'].sum().reset_index()
    
    channel_monthly['year'] = channel_monthly['year_month_str'].str[:4]
    
    # Calculate yearly totals
    yearly = channel_monthly.groupby('year')['quantity'].sum()
    
    # Get forecast total
    forecast_total = forecast_df[forecast_df['channel'] == channel]['forecast_qty'].sum()
    
    summary_by_channel.append({
        'Channel': channel,
        '2022_Total': int(yearly.get('2022', 0)),
        '2023_Total': int(yearly.get('2023', 0)),
        '2024_Total': int(yearly.get('2024', 0)),
        '2025_Total': int(yearly.get('2025', 0)),
        '2026_YTD': int(yearly.get('2026', 0)),
        '2026_Forecast': int(forecast_total)
    })

summary_df = pd.DataFrame(summary_by_channel)
summary_df['YoY_Growth'] = ((summary_df['2026_Forecast'] - summary_df['2025_Total']) / summary_df['2025_Total'] * 100).round(1)

print("\nChannel Summary:")
summary_df

In [ ]:
# Create pivot tables for each channel
pivot_tables = {}

for channel in all_channels:
    # Combine historical and forecast
    if channel == 'TOTAL':
        hist_data = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()
    else:
        hist_data = monthly_data[monthly_data['orders__source'] == channel].copy()
        hist_data = hist_data[['year_month_str', 'products__variants__sku', 'quantity']]
    
    # Get forecast for this channel
    fcst_data = forecast_df[forecast_df['channel'] == channel][['month', 'sku', 'forecast_qty']].copy()
    fcst_data.columns = ['year_month_str', 'products__variants__sku', 'quantity']
    
    # Combine
    combined = pd.concat([hist_data, fcst_data], ignore_index=True)
    
    # Pivot
    pivot = combined.pivot_table(
        index='products__variants__sku',
        columns='year_month_str',
        values='quantity',
        fill_value=0,
        aggfunc='sum'
    )
    
    pivot_tables[channel] = pivot

print("Pivot tables created for all channels")

In [ ]:
# Authenticate with Google
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

print("Authenticated with Google")

In [ ]:
# Create new Google Sheet
sheet_name = f"Demand_Planning_{datetime.now().strftime('%Y%m%d_%H%M')}"
sh = gc.create(sheet_name)

print(f"Created Google Sheet: {sheet_name}")
print(f"URL: https://docs.google.com/spreadsheets/d/{sh.id}")

In [ ]:
# Add Summary sheet
summary_sheet = sh.sheet1
summary_sheet.update_title('Summary Dashboard')

# Write summary data
summary_sheet.update('A1', [['DEMAND PLANNING SUMMARY']])
summary_sheet.update('A3', [['Historical Period: 2022-01 through 2026-01']])
summary_sheet.update('A4', [['Forecast Period: 2026-02 through 2026-12']])
summary_sheet.update('A5', [['Method: 12-Month Moving Average with Seasonal Adjustment']])

# Write channel summary
summary_sheet.update('A7', [['CHANNEL SUMMARY']])
summary_data = [summary_df.columns.tolist()] + summary_df.values.tolist()
summary_sheet.update('A9', summary_data)

# Write forecast comparison
summary_sheet.update('A16', [['FORECAST AGGREGATION COMPARISON']])
comparison_data = [comparison_df.columns.tolist()] + comparison_df.values.tolist()
summary_sheet.update('A18', comparison_data)

print("Summary Dashboard created")

In [ ]:
# Add Product Type Breakdown sheet
pt_sheet = sh.add_worksheet(title='Product Type Breakdown', rows=1000, cols=20)

pt_sheet.update('A1', [['FORECAST BY PRODUCT TYPE']])
pt_data = [product_type_df.columns.tolist()] + product_type_df.values.tolist()
pt_sheet.update('A3', pt_data)

print("Product Type Breakdown sheet created")

In [ ]:
# Add forecast sheets for each channel
for channel in all_channels:
    print(f"Creating sheet for {channel}...")
    
    # Create worksheet
    ws = sh.add_worksheet(title=f"{channel} - Forecast", rows=5000, cols=100)
    
    # Get forecast data
    channel_forecast = forecast_df[forecast_df['channel'] == channel].copy()
    
    # Write data
    ws.update('A1', [[f'{channel} - Monthly Forecast']])
    forecast_data = [channel_forecast.columns.tolist()] + channel_forecast.values.tolist()
    ws.update('A3', forecast_data)
    
    # Add pivot view
    ws_pivot = sh.add_worksheet(title=f"{channel} - Pivot", rows=1000, cols=100)
    pivot = pivot_tables[channel]
    
    # Convert pivot to list format
    pivot_data = [['SKU'] + pivot.columns.tolist()]
    for idx in pivot.index:
        pivot_data.append([idx] + pivot.loc[idx].tolist())
    
    ws_pivot.update('A1', pivot_data)

print("All channel sheets created")

In [ ]:
# Add Aggregated By Year sheet (formerly Trend Analysis)
print("Creating Aggregated By Year analysis...")

trend_data = []
for sku in sku_details['products__variants__sku'].unique():
    sku_info = sku_details[sku_details['products__variants__sku'] == sku].iloc[0]
    
    # Get TOTAL channel data
    total_data = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()
    sku_data = total_data[total_data['products__variants__sku'] == sku].copy()
    sku_data['year'] = sku_data['year_month_str'].str[:4]
    
    yearly = sku_data.groupby('year')['quantity'].sum()
    
    trend_data.append({
        'SKU': sku,
        'Product': sku_info['products__variants__title'],
        'Product_Type': sku_info['products__product_type'],
        '2022': int(yearly.get('2022', 0)),
        '2023': int(yearly.get('2023', 0)),
        '2024': int(yearly.get('2024', 0)),
        '2025': int(yearly.get('2025', 0)),
        '2026_YTD': int(yearly.get('2026', 0))
    })

trend_df = pd.DataFrame(trend_data)

# Create worksheet
ws_trend = sh.add_worksheet(title="Aggregated By Year", rows=1000, cols=20)
trend_sheet_data = [trend_df.columns.tolist()] + trend_df.values.tolist()
ws_trend.update('A1', trend_sheet_data)

print("Aggregated By Year sheet created")

In [ ]:
# Format the summary sheet (basic formatting)
summary_sheet.format('A1', {
    'textFormat': {'bold': True, 'fontSize': 14}
})

summary_sheet.format('A7', {
    'textFormat': {'bold': True, 'fontSize': 12}
})

summary_sheet.format('A16', {
    'textFormat': {'bold': True, 'fontSize': 12}
})

# Google Blue header color (#4285f4)
summary_sheet.format('A9:H9', {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
})

summary_sheet.format('A18:F18', {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
})

print("Formatting applied")

In [ ]:
# Final output
print("\n" + "="*80)
print("DEMAND PLANNING TOOL CREATED SUCCESSFULLY!")
print("="*80)
print(f"\nGoogle Sheet Name: {sheet_name}")
print(f"URL: https://docs.google.com/spreadsheets/d/{sh.id}")
print(f"\nSheets created:")
for worksheet in sh.worksheets():
    print(f"  - {worksheet.title}")
print(f"\nTotal SKUs analyzed: {len(sku_details)}")
print(f"Channels: {', '.join(all_channels)}")
print(f"Forecast period: Feb 2026 - Dec 2026")
print(f"\nKey Features:")
print(f"  - Forecast aggregation comparison (Channel, Product Type, SKU levels)")
print(f"  - Product type breakdown with mix percentages")
print(f"  - Historical data aggregated by year")
print(f"\nYou can now open the Google Sheet and start using it for production planning!")